In [0]:
updates_df = spark.table("finance_fraudworkspace.silver_managed.accounts_silver")

In [0]:
# SDC TYPE 2 LOGIC
from delta.tables import DeltaTable

silver_accounts_table = "finance_fraudworkspace.silver_managed.accounts_silver"
silver_table = DeltaTable.forName(spark, silver_accounts_table)

In [0]:
spark.sql("""
UPDATE finance_fraudworkspace.silver_managed.accounts_silver
SET 
  effective_start_date = current_timestamp(),
  is_current = true
WHERE is_current IS NULL
""")

DataFrame[num_affected_rows: bigint]

In [0]:
from pyspark.sql.functions import current_timestamp, lit

updates_df = updates_df \
    .withColumn("effective_start_date", current_timestamp()) \
    .withColumn("effective_end_date", lit(None).cast("timestamp")) \
    .withColumn("is_current", lit(True))

In [0]:
(
    silver_table.alias("target")
    .merge(
        updates_df.alias("source"),
        "target.Account_id = source.Account_id AND target.is_current = true"
    )
    # If values changed → expire old row
    .whenMatchedUpdate(
        condition="""
            target.Account_status <> source.Account_status OR
            target.Credit_limit <> source.Credit_limit OR
            target.Updated_ts <> source.Updated_ts
        """,
        set={
            "effective_end_date": "current_timestamp()",
            "is_current": "false"
        }
    )

    .whenNotMatchedInsertAll()
    .execute()
)

DataFrame[num_affected_rows: bigint, num_updated_rows: bigint, num_deleted_rows: bigint, num_inserted_rows: bigint]

In [0]:
# VERIFY THE SDC TYPE 2 LOGIC
display(
    spark.table("finance_fraudworkspace.silver_managed.accounts_silver")
    .orderBy("Account_id", "effective_start_date")
)

Customer_id,Account_id,Account_status,Credit_limit,Updated_ts,effective_start_date,effective_end_date,is_current
C001,A001,ACTIVE,10000,2024-01-01T00:00:00Z,2026-01-29T20:21:38.461099Z,null,true
C002,A002,BLOCKED,5000,2024-02-15T00:00:00Z,2026-01-29T20:21:38.461099Z,null,true
C003,A003,BLOCKED,3000,2024-01-01T00:00:00Z,2026-01-29T20:21:38.461099Z,null,true
C004,A004,ACTIVE,1000,2024-01-01T00:00:00Z,2026-01-29T20:21:38.461099Z,null,true
C005,A005,ACTIVE,7000,2024-01-01T00:00:00Z,2026-01-29T20:21:38.461099Z,null,true


In [0]:
%sql
DESCRIBE HISTORY finance_fraudworkspace.silver_managed.accounts_silver;

version,timestamp,userId,userName,operation,operationParameters,job,notebook,clusterId,readVersion,isolationLevel,isBlindAppend,operationMetrics,userMetadata,engineInfo
3,2026-01-29T20:27:25Z,149160287736310,x23122498-nci@outlook.com,MERGE,"Map(predicate -> [""((Account_id#291437 = Account_id#262923) AND is_current#291443)""], clusterBy -> [], matchedPredicates -> [{""predicate"":""((NOT (Account_status#291438 = Account_status#262924) OR NOT (Credit_limit#291439 = Credit_limit#262925)) OR NOT (Updated_ts#291440 = Updated_ts#262926))"",""actionType"":""update""}], statsOnLoad -> false, notMatchedBySourcePredicates -> [], notMatchedPredicates -> [{""actionType"":""insert""}])",null,List(1632028177295827),0124-183127-on88zhrb,2,WriteSerializable,false,"Map(numTargetRowsCopied -> 0, numTargetRowsDeleted -> 0, numTargetFilesAdded -> 0, numTargetBytesAdded -> 0, numTargetBytesRemoved -> 0, numTargetDeletionVectorsAdded -> 0, numTargetRowsMatchedUpdated -> 0, executionTimeMs -> 2736, materializeSourceTimeMs -> 1, numTargetRowsInserted -> 0, numTargetRowsMatchedDeleted -> 0, numTargetDeletionVectorsUpdated -> 0, scanTimeMs -> 1617, numTargetRowsUpdated -> 0, numOutputRows -> 0, numTargetDeletionVectorsRemoved -> 0, numTargetRowsNotMatchedBySourceUpdated -> 0, numTargetChangeFilesAdded -> 0, numSourceRows -> 5, numTargetFilesRemoved -> 0, numTargetRowsNotMatchedBySourceDeleted -> 0, rewriteTimeMs -> 1074)",null,Databricks-Runtime/17.3.x-scala2.13
2,2026-01-29T20:21:42Z,149160287736310,x23122498-nci@outlook.com,UPDATE,"Map(predicate -> [""isnull(is_current#283171)""])",null,List(1632028177295827),0124-183127-on88zhrb,1,WriteSerializable,false,"Map(numRemovedFiles -> 1, numRemovedBytes -> 1972, numCopiedRows -> 0, numDeletionVectorsAdded -> 0, numDeletionVectorsRemoved -> 0, numAddedChangeFiles -> 0, executionTimeMs -> 2857, numDeletionVectorsUpdated -> 0, scanTimeMs -> 1648, numAddedFiles -> 1, numUpdatedRows -> 5, numAddedBytes -> 2874, rewriteTimeMs -> 1197)",null,Databricks-Runtime/17.3.x-scala2.13
1,2026-01-29T20:18:23Z,149160287736310,x23122498-nci@outlook.com,ADD COLUMNS,"Map(columns -> [{""column"":{""name"":""effective_start_date"",""type"":""timestamp"",""nullable"":true,""metadata"":{}}},{""column"":{""name"":""effective_end_date"",""type"":""timestamp"",""nullable"":true,""metadata"":{}}},{""column"":{""name"":""is_current"",""type"":""boolean"",""nullable"":true,""metadata"":{}}}])",null,List(1632028177295827),0124-183127-on88zhrb,0,WriteSerializable,true,Map(),null,Databricks-Runtime/17.3.x-scala2.13
0,2026-01-29T20:02:28Z,149160287736310,x23122498-nci@outlook.com,CREATE OR REPLACE TABLE AS SELECT,"Map(partitionBy -> [], clusterBy -> [], description -> null, isManaged -> true, properties -> {""delta.parquet.compression.codec"":""zstd"",""delta.enableDeletionVectors"":""true""}, statsOnLoad -> false)",null,List(1632028177295826),0124-183127-on88zhrb,null,WriteSerializable,false,"Map(numFiles -> 1, numRemovedFiles -> 0, numRemovedBytes -> 0, numDeletionVectorsRemoved -> 0, numOutputRows -> 5, numOutputBytes -> 1972)",null,Databricks-Runtime/17.3.x-scala2.13
